**Hidden Markov models for cracking codes**

In this exercise you have to make a partially built HMM work and use it to solve some simple substitution ciphers. Plaintext data is provided in 'plaintext' directory. Encrypted data is in 'encrypted'. Some of the texts were originally English some of them were Russian; the sequences are also of different lengths. 

This homework is worth **15 points** and is due by the next class (**24th Oct.**), please submit the results of the **TASK 5** (a list of files and names of the author/work) to Anytask in the following format: 'filename author' where 'filename' is a file from "encrypted/\*_encrypted.txt" and 'author' is a file from "plaintext/\*.txt" (not including 'english.txt', 'russian.txt' or 'all.txt') which best matches the decrypted text.




In [1]:
# Utilities for loading data from file and converting characters to integers and back.
import numpy as np
from tqdm import tqdm_notebook as tqdm
    
def get_char_to_int_mapping(path):
    # Load data from path and get mapping from characters to integers and back.
    characters = set()
    for line in open(path):
        characters.update(set([c for c in line.strip()]))
    char_to_int_mapping = dict([(char, i) for i, char in enumerate(sorted(list(characters)))])
    int_to_char_mapping = [char for char, i in char_to_int_mapping.items()]
    return char_to_int_mapping, int_to_char_mapping

def load_sequences(path, char_to_int_mapping):
    # Load data from path and map to integers using mapping.
    return [[char_to_int_mapping[c] for c in line.strip()] for line in open(path)]

def estimate_markov_model_from_sequences(sequences, num_states):
    # Estimate a Markov model based on the sequences (integers) provided.

    # pi[i] = Pr(s_0 = i)
    pi_counts = np.zeros(num_states)

    # A[i, j] = Pr(s_t = j | s_{t-1} = i)
    A_counts = np.zeros((num_states, num_states))
    
    for n, sequence in enumerate(sequences):
        prev_char = None
        for n_char, char in enumerate(sequence):
            if n_char == 0:
                pi_counts[char] += 1
            
            if prev_char is not None:
                A_counts[prev_char, char] += 1
            
            prev_char = char
            
    pi = pi_counts / np.sum(pi_counts)
    A = A_counts / np.sum(A_counts, 1, keepdims=True)
    
    return pi, A

**TASK 1**: Make the following block run by completing the method 'estimate_markov_model_from_sequences' above.

In [2]:
# Some data to use.
plaintext = 'plaintext/english.txt'
# plaintext = 'plaintext/shakespeare.txt'
# plaintext = 'plaintext/russian.txt'

ciphertext = 'encrypted/1_encrypted.txt' # short sequences in english
# ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)

# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))

In [3]:
prev_char, next_char = np.unravel_index(A.argmax(), A.shape)
int_to_char_mapping[prev_char], int_to_char_mapping[next_char]

('q', 'u')

Below is a mostly implemented HMM.

$\alpha_{t+1}(i) = \sum_{j} \alpha_{t}(j)A_{j}(i)B_{i}(x_{t+1})$

$\beta_{t}(i) = \sum_{j}\beta_{t+1}A_{i}(j)B_{j}(x_{t+1})$

In [4]:
class HMM_base():
    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}, seed=None,
                 print_test_output=True, early_stopping_thresh=None):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping)
        self.num_outputs = len(observations_to_char_mapping)
        self.states_to_char_mapping = states_to_char_mapping
        self.observations_to_char_mapping = observations_to_char_mapping
        self.print_test_output = print_test_output
        self.early_stopping_thresh = early_stopping_thresh
       
        if seed is not None:
            np.random.seed(seed)
        # Random initialization
        self.pi = np.random.rand(self.num_states)
        self.pi /= np.sum(self.pi)
        
#         self.pi = np.ones((self.num_states)) / self.num_states
        
        self.A = np.random.rand(self.num_states, self.num_states)
        self.A /= np.sum(self.A, 1, keepdims=True)
        
#         self.A = np.ones((self.num_states, self.num_states)) / self.num_states
#       
        
        self.B = np.ones((self.num_states, self.num_outputs)) / self.num_outputs
        self.B = self.B + np.random.rand(self.num_states, self.num_outputs) / 100
        self.B = np.clip(self.B, 0, None)
#         self.B = np.random.rand(self.num_states, self.num_outputs)
        self.B /= np.sum(self.B, 1, keepdims=True)
        
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        if self.fixed_pi:
            print('pi is fixed')
            self.pi = parameters['pi']
        
        self.fixed_A = 'A' in parameters
        if self.fixed_A:
            print('A is fixed')
            self.A = parameters['A']
            
        self.fixed_B = 'B' in parameters
        if self.fixed_B:
            print('B is fixed')
            self.B = parameters['B']
    
        previous_llh = None
        iter = 0
        while True and iter < max_iters:
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences)

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts)

            # Output some sequences for debugging.
            if self.print_test_output:
                self.output(sequences[:10])

            # Log likelihood should be increasing
            print('iteration %d; log likelihood %.4f' % (iter, log_likelihood))
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
                if self.early_stopping_thresh is not None:
                    if abs(previous_llh - log_likelihood) < self.early_stopping_thresh:
                        print(f'early stopping previous_llh {previous_llh}, log_likelihood {log_likelihood}')
                        break
                        
            previous_llh = log_likelihood
        
            iter += 1


    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            # Run Forward-Backward dynamic program
            alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sequence)
  
            # Accumulate statistics
            pi_counts += gamma[:, 0]
            A_counts += xi
            for t, x in enumerate(sequence):
                B_counts[:, x] += gamma[:, t]
            
            total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))

In [5]:
# divide classes for clarity
class HMM(HMM_base):        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha, alpha_norms = self.forward(sequence)
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence, alpha_norms)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        gamma = (alpha * beta) / np.sum(alpha * beta, 0)# * (alpha_norms.T ** 2)
        # gamma is normalized -> no need to apply alpha_norms

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = alpha[:, t].reshape(-1, 1) * self.A * beta[:, t+1] * self.B[:, sequence[t+1]]
            xi += this_xi / np.sum(this_xi)
        
        # * alpha_norms[len(sequence)-1]
        log_likelihood = np.log(np.sum(alpha[:, len(sequence)-1]))
        
        if alpha_norms is not None:
            log_likelihood = np.sum(np.log(alpha_norms))
        
        return alpha, beta, gamma, xi, log_likelihood
    
    def forward(self, sequence, scale=True):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)))
        alpha_norms = np.zeros(len(sequence)) if scale else None
        
        for t, xt in enumerate(sequence):
            if t == 0:
                prev_alpha = self.pi * self.B[:, xt]
                alpha_initialized = True
            else:
                prev_alpha = self.A.T.dot(prev_alpha).flatten() * self.B[:, xt]
            
            alpha_norm = np.sum(prev_alpha)
            
            if scale:
                prev_alpha = prev_alpha / alpha_norm
                alpha_norms[t] = alpha_norm
                
            alpha[:, t] = prev_alpha.copy()
            
        
        assert(alpha_initialized)
        return alpha, alpha_norms
    
    def backward(self, sequence, alpha_norms=None):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        num_states = len(self.pi)
        beta = np.zeros((num_states, len(sequence)))
    
        for ind, xt in enumerate(sequence[::-1]):
            t = len(sequence) - ind - 1
            if ind == 0:
                prev_beta = np.ones(num_states)
                beta_initialized = True
            else:
                prev_beta = (prev_beta * self.B[:, xt_plus1]).dot(self.A.T)                
            
            if alpha_norms is not None:
                prev_beta = prev_beta / alpha_norms[t]
            beta[:, t] = prev_beta.copy()
            xt_plus1 = xt
    
        assert(beta_initialized)
        
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))
    
    def decode(self, sequences):
        decoded_tokens = []
        decoded_text = []
        
        for sequence in sequences:
            tokens = self.max_posterior_decode(sequence)
            text = [self.states_to_char_mapping[x] for x in tokens]
            decoded_tokens.append(tokens)
            decoded_text.append(''.join(text))
        
        return decoded_tokens, decoded_text

Compare numpy and cycles

**TASK 2**: Implement the assertions in 'forward' and 'backward' methods on the HMM class so that the following block passes.

In [6]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping

# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping, seed=17, print_test_output=False, early_stopping_thresh=1)

# Estimate the parameters and decode the encrypted sequences.
hmm.estimate_with_em(encrypted_sequences[:100],
                     parameters={'A':A, 'pi':pi},
                    )

pi is fixed
A is fixed
iteration 0; log likelihood -12087.8863
iteration 1; log likelihood -10425.6340
iteration 2; log likelihood -10414.1728
iteration 3; log likelihood -10400.7130
iteration 4; log likelihood -10382.3546
iteration 5; log likelihood -10354.5472
iteration 6; log likelihood -10310.6183
iteration 7; log likelihood -10242.5469
iteration 8; log likelihood -10143.6626
iteration 9; log likelihood -10010.5171
iteration 10; log likelihood -9841.0102
iteration 11; log likelihood -9639.8194
iteration 12; log likelihood -9435.3972
iteration 13; log likelihood -9262.8650
iteration 14; log likelihood -9127.5356
iteration 15; log likelihood -9016.7499
iteration 16; log likelihood -8921.8371
iteration 17; log likelihood -8842.3876
iteration 18; log likelihood -8779.3327
iteration 19; log likelihood -8730.7606
iteration 20; log likelihood -8693.5743
iteration 21; log likelihood -8665.1298
iteration 22; log likelihood -8643.4481
iteration 23; log likelihood -8626.8097
iteration 24; log

In [7]:
hmm.output(encrypted_sequences[:10])

(states):       than such a roman
(observations): noeixjtcoxexhwfei
(states):       cassous brutus bait not me
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       ill not endure it you forget yourself
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       to hedge me in i am a soldier i
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       older in prackice abler than yourself
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       to mave conditions
(observations): nwxfeaqxcwi gngwij
(states):       brutus go to you are not cassous
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       cassous i am
(observations): cejjgtjxgxef
(states):       brutus i say you are not
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       cassous urge me no more i thall forget myself
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp


**TASK 3**: Some of the encrypted sequences are quite long. Try decoding some from 'encrypted/99_encrypted.txt' (note these are in Russian).

In [8]:
# Some data to use.
plaintext = 'plaintext/russian.txt'

ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)

# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))

In [9]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping

# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping, seed=None, print_test_output=False, early_stopping_thresh=1)

# Estimate the parameters and decode the encrypted sequences.
hmm.estimate_with_em(encrypted_sequences[:100],
                     parameters={'A':A, 'pi':pi},
                    )

pi is fixed
A is fixed
iteration 0; log likelihood -75849.1413
iteration 1; log likelihood -61365.4920
iteration 2; log likelihood -61360.6204
iteration 3; log likelihood -61352.4391
iteration 4; log likelihood -61336.7049
iteration 5; log likelihood -61304.1466
iteration 6; log likelihood -61234.8371
iteration 7; log likelihood -61105.8987
iteration 8; log likelihood -60912.6211
iteration 9; log likelihood -60639.7005
iteration 10; log likelihood -60276.3419
iteration 11; log likelihood -59842.6999
iteration 12; log likelihood -59374.5491
iteration 13; log likelihood -58862.5497
iteration 14; log likelihood -58271.5384
iteration 15; log likelihood -57569.3887
iteration 16; log likelihood -56771.8011
iteration 17; log likelihood -55964.2246
iteration 18; log likelihood -55231.4917
iteration 19; log likelihood -54604.1579
iteration 20; log likelihood -54072.0722
iteration 21; log likelihood -53633.9813
iteration 22; log likelihood -53288.0677
iteration 23; log likelihood -52985.9379
ite

In [10]:
hmm.output(encrypted_sequences[:10])

(states):       19286 андрей пожал плечами и поморщился как морщатся любители музыки услышав пальшивую ноту обе женщины отпустили друг друга потом опять как бучто боясь опоздать схватили друг друга за руки стали целовать и отрывать руки и потом опять стали целовать друг друга в лицо и совершенно неожиданно для 19280 андрея обе заплакали и опять стали целоваться тоже заплакала 19234 андрею было очевидно неловко но для двух женщин казалось так естественно что они плакали казалось они и не предполагали чтобы могло иначе совершиться это свидание
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

**TASK 4**: Make your implementation of forward and backward more efficient by removing all but the outermost for-loop.

Done, version with loops is moved to the bottom of this notebook (HMM_cycles)

**TASK 5**: Try to classify the author of each text. 

#### Lets first try to create bilingual model: joined russian + english mapping + A,pi

In [11]:
russian_plaintext = 'plaintext/russian.txt'
english_plaintext = 'plaintext/english.txt'
                                                                                   
ru_char_to_int_mapping, ru_int_to_char_mapping = get_char_to_int_mapping(russian_plaintext)
en_char_to_int_mapping, en_int_to_char_mapping = get_char_to_int_mapping(english_plaintext)

In [12]:
char_to_int_mapping = {char: int_code + len(ru_char_to_int_mapping) - 1 # space is common
                       for char, int_code in en_char_to_int_mapping.items()}

char_to_int_mapping.update(ru_char_to_int_mapping)
int_to_char_mapping = ru_int_to_char_mapping + en_int_to_char_mapping[1:]

In [13]:
for char, int_code in char_to_int_mapping.items():
    assert(int_to_char_mapping[int_code] == char)

#### Load sequences and create joined pi and A

In [14]:
plaintext_sequences = load_sequences(russian_plaintext, char_to_int_mapping)
plaintext_sequences += load_sequences(english_plaintext, char_to_int_mapping)

# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))

In [15]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping

# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping, seed=17, print_test_output=False, early_stopping_thresh=1)

# Estimate the parameters and decode the encrypted sequences.
hmm.estimate_with_em(encrypted_sequences[:100],
                     parameters={'A':A, 'pi':pi},
                     max_iters=200
                    )

pi is fixed
A is fixed
iteration 0; log likelihood -85362.0392
iteration 1; log likelihood -61357.5870
iteration 2; log likelihood -61340.4636
iteration 3; log likelihood -61299.6340
iteration 4; log likelihood -61191.9074
iteration 5; log likelihood -60909.5046
iteration 6; log likelihood -60296.5942
iteration 7; log likelihood -59390.8230
iteration 8; log likelihood -58396.9388
iteration 9; log likelihood -57405.9911
iteration 10; log likelihood -56472.1195
iteration 11; log likelihood -55677.8974
iteration 12; log likelihood -55042.2895
iteration 13; log likelihood -54529.0451
iteration 14; log likelihood -54099.5316
iteration 15; log likelihood -53730.2001
iteration 16; log likelihood -53408.6185
iteration 17; log likelihood -53126.9351
iteration 18; log likelihood -52879.6729
iteration 19; log likelihood -52664.5111
iteration 20; log likelihood -52480.8957
iteration 21; log likelihood -52327.4292
iteration 22; log likelihood -52200.8964
iteration 23; log likelihood -52096.8740
ite

In [16]:
hmm.output(encrypted_sequences[:10])

(states):       thand ingher пожал плечами y поморщился wit морщатся любители муnыки услышав пальшивую haly ace женщины отпустили than друга потом aпald wit бучто боясь опоздать схватили than друга за руки стали целовать y отрывать руки y потом aпald стали целовать than друга s лицо y совершенно неожиданно дrя thand inghed ace заплакали y aпald стали целоваться тave заплакала thand inghed было очевидно неrовко но дrя двух жungon казалось lit естественно что any плакали казалось any y he предполагали чтобы могло oнаve совершиться это свидание
(observations): йдрьуопдшщмзояю4пфояфм3пч о ояючющ8 ф0ройпйочющ8пъ0рофцх ъмф очнь2й он0ф2гпвоипфуг внцодюъноюхмо4мд8 д2оюъян0ъ ф ошщн1ошщн1пояюъючоюяръуойпйохншъюохюр0уоюяюьшпъуо0твпъ ф ошщн1ошщн1поьпощнй о0ъпф о5мфювпъуо оюъщ2впъуощнй о ояюъючоюяръуо0ъпф о5мфювпъуошщн1ошщн1повоф 5юо о0ювмщгмддюодмю4 шпддюошфройдрьропдшщмроюхмоьпяфпйпф о оюяръуо0ъпф о5мфювпъу0роъю4моьпяфпйпфпойдрьцопдшщмцох2фюою3мв шдюодмфювйюодюошфрошвнто4мд8 дойпьпфю0уоъпйом0ъм0ъ

#### Quality is worse than using separate eng / russian models. Better stick to separate models, because about 30 percent of decoded letters are in the wrong language, and this will severely affect cosine similarity.

In [23]:
from pathlib import Path
plaintexts_path = Path('plaintext/')

In [24]:
def get_pi_A(plaintext):
    char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)
    plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
    pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))
    
    return pi, A, char_to_int_mapping, int_to_char_mapping

In [25]:
language_texts = ['english.txt', 'russian.txt']

In [26]:
from collections import defaultdict

In [27]:
params_dict = defaultdict(dict)
for plain_text_file in plaintexts_path.iterdir():
    if plain_text_file.name in language_texts:
        pi, A, char_to_int_mapping, int_to_char_mapping = get_pi_A(plain_text_file)
        language = plain_text_file.name.strip('.txt')
        params_dict[language]['pi'] = pi
        params_dict[language]['A'] = A
        params_dict[language]['char_to_int_mapping'] = char_to_int_mapping
        params_dict[language]['int_to_char_mapping'] = int_to_char_mapping

In [28]:
params_dict.keys()

dict_keys(['english', 'russian'])

In [29]:
def bilingual_load_sequences(plain_text_file):
    for language in ['russian', 'english']:
        char_to_int_mapping = params_dict[language]['char_to_int_mapping']
        try:
            sequences = load_sequences(plain_text_file, char_to_int_mapping)
            return sequences, language
        except KeyError:
            continue

In [184]:
plaintexts = defaultdict(list)

for plain_text_file in tqdm(list(plaintexts_path.iterdir())):
    if plain_text_file.suffix != '.txt':
        continue
    if plain_text_file.name in ['all.txt', 'russian.txt', 'english.txt']: #no point to consider it a class
        continue
    
    sequences, language = bilingual_load_sequences(plain_text_file)
    original_text = open(plain_text_file).read().splitlines()
    plaintexts[language].append((sequences, plain_text_file.name, original_text))

In [186]:
def decode(encrypted_sequences, language, n=100):
    pi = params_dict[language]['pi']
    A = params_dict[language]['A']
    int_to_char_mapping = params_dict[language]['int_to_char_mapping']
    
    state_to_char_mapping = int_to_char_mapping
    observation_to_char_mapping = int_to_char_mapping

    # Initialize a HMM with the correct state/output spaces.
    hmm = HMM(observation_to_char_mapping, state_to_char_mapping, seed=17,
              print_test_output=False, early_stopping_thresh=1)

    # Estimate the parameters and decode the encrypted sequences.
    hmm.estimate_with_em(encrypted_sequences[:n],
                         parameters={'A':A, 'pi':pi},
                         max_iters=100
                        )
    
    decoded_tokens, decoded_text = hmm.decode(encrypted_sequences[:n])
    return decoded_tokens, decoded_text

#### Let's compare sentences by calculating cosine similarity between decoded and plain.
Dimensions have to allow dot to happen -> we will trim / pad them.

In [258]:
from numpy.linalg import norm

def merge_sequences(text):
    return [char for sent in text for char in sent]

def trim_for_dot(plain, decoded_len):
    extra_len = len(plain) % decoded_len
    
    if extra_len > 0:
        plain = plain[:-extra_len]
    plain = np.array(plain).reshape(-1, decoded_len)
    return plain
    
def pad_for_dot((plain, decoded_len):
    if len(plain) % decoded_len != 0:
        a = decoded_len - (len(plain) % decoded_len)
        plain = np.pad(plain, (0, a), constant_values=-1)
    
    plain = np.array(plain).reshape(-1, decoded_len)
    
    return plain

In [265]:
def max_cosine_sim(decoded_tokens, text):
    decoded_tokens_merged = merge_sequences(decoded_tokens)
    concatenated_text = merge_sequences(text)
    
    if len(decoded_tokens_merged) == 0 or len(concatenated_text) == 0:
        return 0
    
    if len(decoded_tokens_merged) <= len(concatenated_text):
        min_len_text = decoded_tokens_merged
        max_len_text = concatenated_text
    else:
        min_len_text = concatenated_text
        max_len_text = decoded_tokens_merged
    
    max_len_text = trim_for_dot(max_len_text, len(min_len_text))

    cos_sim = np.dot(min_len_text,  max_len_text.T)/(norm(min_len_text) * norm(max_len_text, axis=1))
    
    return max(cos_sim)

assert(max_cosine_sim([[1,2,3,4]],[[1,2,3,4],[4,3,2,1]]) == 1.0)

In [296]:
def find_origin(decoded_tokens, language, n_sents=3):
    max_cosine = 0
    mle_textfile = None
    max_sents = None

    for text, textfile, original_text in plaintexts[language]:
        for start_ind in tqdm(range(0, len(text))):
            compared_sents = text[start_ind:start_ind + n_sents]

            cosine = max_cosine_sim(decoded_tokens[:n_sents], compared_sents)

            if cosine > max_cosine:
                max_cosine = cosine
                mle_textfile = textfile
                max_sents = original_text[start_ind:start_ind + n_sents]

    return mle_textfile

In [300]:
encrypted_path = Path('encrypted/')
decrypted_path = Path('decrypted/')

In [314]:
origins = []
for ciphertext in tqdm(list(encrypted_path.iterdir())):
    if ciphertext.suffix != '.txt':
        continue
    
    encrypted_sequences, language = bilingual_load_sequences(ciphertext)
    decoded_tokens, decoded_text = decode(encrypted_sequences, language)
    
    output_name = ciphertext.name.replace('en', 'de')
    output_path = decrypted_path / output_name
    with open(output_path, 'w+') as f:
        f.write('\n'.join(decoded_text))
    
    origin_file = find_origin(decoded_tokens, language)
    origins.append((ciphertext.name, origin_file))

pi is fixed
A is fixed
iteration 0; log likelihood -8139.7568
iteration 1; log likelihood -6663.3704
iteration 2; log likelihood -6657.3548
iteration 3; log likelihood -6648.7506
iteration 4; log likelihood -6634.1730
iteration 5; log likelihood -6607.6328
iteration 6; log likelihood -6565.0294
iteration 7; log likelihood -6503.3267
iteration 8; log likelihood -6420.4731
iteration 9; log likelihood -6315.3126
iteration 10; log likelihood -6187.3436
iteration 11; log likelihood -6051.2568
iteration 12; log likelihood -5935.4445
iteration 13; log likelihood -5851.4574
iteration 14; log likelihood -5793.2697
iteration 15; log likelihood -5750.2670
iteration 16; log likelihood -5715.0656
iteration 17; log likelihood -5683.6691
iteration 18; log likelihood -5652.6587
iteration 19; log likelihood -5623.4798
iteration 20; log likelihood -5597.8667
iteration 21; log likelihood -5575.7508
iteration 22; log likelihood -5556.9034
iteration 23; log likelihood -5541.1443
iteration 24; log likelihoo

pi is fixed
A is fixed
iteration 0; log likelihood -13835.0008
iteration 1; log likelihood -11803.9745
iteration 2; log likelihood -11800.8107
iteration 3; log likelihood -11797.0864
iteration 4; log likelihood -11792.2567
iteration 5; log likelihood -11785.4276
iteration 6; log likelihood -11775.1050
iteration 7; log likelihood -11758.8659
iteration 8; log likelihood -11733.1358
iteration 9; log likelihood -11693.6151
iteration 10; log likelihood -11637.0119
iteration 11; log likelihood -11563.1911
iteration 12; log likelihood -11473.9893
iteration 13; log likelihood -11366.5734
iteration 14; log likelihood -11228.0707
iteration 15; log likelihood -11047.5720
iteration 16; log likelihood -10841.6040
iteration 17; log likelihood -10641.7374
iteration 18; log likelihood -10467.5122
iteration 19; log likelihood -10324.5674
iteration 20; log likelihood -10207.9203
iteration 21; log likelihood -10113.0530
iteration 22; log likelihood -10038.5730
iteration 23; log likelihood -9982.3234
iter

pi is fixed
A is fixed
iteration 0; log likelihood -13915.8630
iteration 1; log likelihood -12049.3721
iteration 2; log likelihood -12044.8896
iteration 3; log likelihood -12039.3252
iteration 4; log likelihood -12031.6157
iteration 5; log likelihood -12020.0880
iteration 6; log likelihood -12002.1987
iteration 7; log likelihood -11974.4588
iteration 8; log likelihood -11932.9325
iteration 9; log likelihood -11874.5450
iteration 10; log likelihood -11798.6000
iteration 11; log likelihood -11707.6214
iteration 12; log likelihood -11607.7490
iteration 13; log likelihood -11508.1350
iteration 14; log likelihood -11417.5729
iteration 15; log likelihood -11340.7074
iteration 16; log likelihood -11277.6613
iteration 17; log likelihood -11226.2352
iteration 18; log likelihood -11183.8476
iteration 19; log likelihood -11148.2944
iteration 20; log likelihood -11117.8227
iteration 21; log likelihood -11091.0760
iteration 22; log likelihood -11067.1117
iteration 23; log likelihood -11045.4205
ite

pi is fixed
A is fixed
iteration 0; log likelihood -11774.8198
iteration 1; log likelihood -10271.6799
iteration 2; log likelihood -10264.8700
iteration 3; log likelihood -10257.3051
iteration 4; log likelihood -10247.7274
iteration 5; log likelihood -10234.4159
iteration 6; log likelihood -10214.9908
iteration 7; log likelihood -10186.4338
iteration 8; log likelihood -10145.4130
iteration 9; log likelihood -10088.8388
iteration 10; log likelihood -10014.7667
iteration 11; log likelihood -9923.7280
iteration 12; log likelihood -9820.1934
iteration 13; log likelihood -9713.3073
iteration 14; log likelihood -9614.4702
iteration 15; log likelihood -9531.1603
iteration 16; log likelihood -9464.4920
iteration 17; log likelihood -9412.3551
iteration 18; log likelihood -9372.0052
iteration 19; log likelihood -9340.7945
iteration 20; log likelihood -9316.4541
iteration 21; log likelihood -9297.2699
iteration 22; log likelihood -9282.0348
iteration 23; log likelihood -9269.8822
iteration 24; lo

pi is fixed
A is fixed
iteration 0; log likelihood -13407.1908
iteration 1; log likelihood -11665.0728
iteration 2; log likelihood -11659.7333
iteration 3; log likelihood -11653.2537
iteration 4; log likelihood -11644.4015
iteration 5; log likelihood -11631.0224
iteration 6; log likelihood -11609.4080
iteration 7; log likelihood -11573.6979
iteration 8; log likelihood -11516.1459
iteration 9; log likelihood -11428.8423
iteration 10; log likelihood -11303.9746
iteration 11; log likelihood -11129.4871
iteration 12; log likelihood -10900.2347
iteration 13; log likelihood -10653.8682
iteration 14; log likelihood -10438.7443
iteration 15; log likelihood -10260.2792
iteration 16; log likelihood -10110.4810
iteration 17; log likelihood -9989.8714
iteration 18; log likelihood -9895.9024
iteration 19; log likelihood -9823.6056
iteration 20; log likelihood -9769.6982
iteration 21; log likelihood -9730.6966
iteration 22; log likelihood -9702.8510
iteration 23; log likelihood -9683.0431
iteration 

...

pi is fixed
A is fixed
iteration 0; log likelihood -13296.6645
iteration 1; log likelihood -11533.4804
iteration 2; log likelihood -11529.1342
iteration 3; log likelihood -11523.9116
iteration 4; log likelihood -11516.8096
iteration 5; log likelihood -11506.1049
iteration 6; log likelihood -11488.8696
iteration 7; log likelihood -11460.5050
iteration 8; log likelihood -11414.9983
iteration 9; log likelihood -11346.9127
iteration 10; log likelihood -11254.0565
iteration 11; log likelihood -11135.6227
iteration 12; log likelihood -10981.9768
iteration 13; log likelihood -10768.2970
iteration 14; log likelihood -10494.2513
iteration 15; log likelihood -10219.0481
iteration 16; log likelihood -9986.6337
iteration 17; log likelihood -9808.0463
iteration 18; log likelihood -9679.4305
iteration 19; log likelihood -9588.8554
iteration 20; log likelihood -9526.4378
iteration 21; log likelihood -9484.2515
iteration 22; log likelihood -9455.7237
iteration 23; log likelihood -9435.9127
iteration 2

pi is fixed
A is fixed
iteration 0; log likelihood -11951.4334
iteration 1; log likelihood -10197.7929
iteration 2; log likelihood -10192.7331
iteration 3; log likelihood -10186.6003
iteration 4; log likelihood -10178.5124
iteration 5; log likelihood -10167.1170
iteration 6; log likelihood -10150.3888
iteration 7; log likelihood -10125.5839
iteration 8; log likelihood -10089.7367
iteration 9; log likelihood -10040.9300
iteration 10; log likelihood -9979.2390
iteration 11; log likelihood -9904.6545
iteration 12; log likelihood -9810.6139
iteration 13; log likelihood -9677.7124
iteration 14; log likelihood -9489.2173
iteration 15; log likelihood -9277.5098
iteration 16; log likelihood -9094.1681
iteration 17; log likelihood -8943.5232
iteration 18; log likelihood -8814.0284
iteration 19; log likelihood -8706.6276
iteration 20; log likelihood -8624.5057
iteration 21; log likelihood -8565.4940
iteration 22; log likelihood -8524.2820
iteration 23; log likelihood -8495.3134
iteration 24; log

pi is fixed
A is fixed
iteration 0; log likelihood -12087.8863
iteration 1; log likelihood -10425.6340
iteration 2; log likelihood -10414.1728
iteration 3; log likelihood -10400.7130
iteration 4; log likelihood -10382.3546
iteration 5; log likelihood -10354.5472
iteration 6; log likelihood -10310.6183
iteration 7; log likelihood -10242.5469
iteration 8; log likelihood -10143.6626
iteration 9; log likelihood -10010.5171
iteration 10; log likelihood -9841.0102
iteration 11; log likelihood -9639.8194
iteration 12; log likelihood -9435.3972
iteration 13; log likelihood -9262.8650
iteration 14; log likelihood -9127.5356
iteration 15; log likelihood -9016.7499
iteration 16; log likelihood -8921.8371
iteration 17; log likelihood -8842.3876
iteration 18; log likelihood -8779.3327
iteration 19; log likelihood -8730.7606
iteration 20; log likelihood -8693.5743
iteration 21; log likelihood -8665.1298
iteration 22; log likelihood -8643.4481
iteration 23; log likelihood -8626.8097
iteration 24; log

pi is fixed
A is fixed
iteration 0; log likelihood -12047.1969
iteration 1; log likelihood -10426.7504
iteration 2; log likelihood -10419.4942
iteration 3; log likelihood -10410.5731
iteration 4; log likelihood -10398.5474
iteration 5; log likelihood -10381.2310
iteration 6; log likelihood -10355.4900
iteration 7; log likelihood -10317.4584
iteration 8; log likelihood -10263.6838
iteration 9; log likelihood -10193.1794
iteration 10; log likelihood -10109.2741
iteration 11; log likelihood -10019.7588
iteration 12; log likelihood -9934.0331
iteration 13; log likelihood -9858.3988
iteration 14; log likelihood -9793.9377
iteration 15; log likelihood -9738.3097
iteration 16; log likelihood -9688.7707
iteration 17; log likelihood -9643.4007
iteration 18; log likelihood -9600.8808
iteration 19; log likelihood -9560.5759
iteration 20; log likelihood -9522.2347
iteration 21; log likelihood -9485.2196
iteration 22; log likelihood -9448.2614
iteration 23; log likelihood -9409.9234
iteration 24; l

pi is fixed
A is fixed
iteration 0; log likelihood -13294.8651
iteration 1; log likelihood -11453.2922
iteration 2; log likelihood -11447.2731
iteration 3; log likelihood -11438.5068
iteration 4; log likelihood -11424.1483
iteration 5; log likelihood -11399.1221
iteration 6; log likelihood -11355.5465
iteration 7; log likelihood -11285.0976
iteration 8; log likelihood -11185.7628
iteration 9; log likelihood -11063.7211
iteration 10; log likelihood -10919.8033
iteration 11; log likelihood -10735.6244
iteration 12; log likelihood -10496.7694
iteration 13; log likelihood -10247.3256
iteration 14; log likelihood -10046.1824
iteration 15; log likelihood -9897.3186
iteration 16; log likelihood -9782.2876
iteration 17; log likelihood -9689.0971
iteration 18; log likelihood -9612.6565
iteration 19; log likelihood -9549.3595
iteration 20; log likelihood -9497.5571
iteration 21; log likelihood -9459.0681
iteration 22; log likelihood -9431.9822
iteration 23; log likelihood -9412.8632
iteration 24

In [327]:
origins = sorted(origins)
origins_str = [f'{enc} {orig}'for enc, orig in origins]

In [328]:
with open('authors.txt', 'w+') as f:
    f.write('\n'.join(origins_str))

In [329]:
!tar -cvzf results.tar.gz decrypted/*.txt

decrypted/0_decrypted.txt
decrypted/100_decrypted.txt
decrypted/101_decrypted.txt
decrypted/102_decrypted.txt
decrypted/103_decrypted.txt
decrypted/104_decrypted.txt
decrypted/105_decrypted.txt
decrypted/106_decrypted.txt
decrypted/107_decrypted.txt
decrypted/108_decrypted.txt
decrypted/109_decrypted.txt
decrypted/10_decrypted.txt
decrypted/110_decrypted.txt
decrypted/111_decrypted.txt
decrypted/112_decrypted.txt
decrypted/113_decrypted.txt
decrypted/114_decrypted.txt
decrypted/115_decrypted.txt
decrypted/116_decrypted.txt
decrypted/117_decrypted.txt
decrypted/118_decrypted.txt
decrypted/119_decrypted.txt
decrypted/11_decrypted.txt
decrypted/120_decrypted.txt
decrypted/121_decrypted.txt
decrypted/122_decrypted.txt
decrypted/123_decrypted.txt
decrypted/124_decrypted.txt
decrypted/125_decrypted.txt
decrypted/126_decrypted.txt
decrypted/127_decrypted.txt
decrypted/128_decrypted.txt
decrypted/129_decrypted.txt
decrypted/12_decrypted.txt
decrypted/130_decrypted.txt
decrypted/131_decrypted.t

#### End of actual hometask, below code was used for debug/testing

---

In [ ]:
assert(False), 'do not go gentle into that good night'

In [ ]:
# sequence = [1, 2, 3, 27]
# num_states = 27
# num_output_states = 28

# alpha = np.random.rand(num_states, len(sequence))
# prev_alpha = np.random.rand(num_states, 1)
# A = np.random.rand(num_states, num_states)
# B = np.random.rand(num_states, num_output_states)

def multiply_non_numpy_way(alpha, A, B, sequence):
    num_states = len(alpha)
    
    res = np.zeros((num_states, len(sequence)))
    for t, xt in enumerate(sequence):
        for i in range(num_states):
            for j in range(num_states):
                res[i, t] += alpha[j, t] * A[i, j] * B[i, xt]
    return res

def multiply_numpy_way(alpha, A, B, sequence):
    num_states = len(alpha)
    
    res = np.zeros((num_states, len(sequence)))
    for t, xt in enumerate(sequence):
        res[:, t] = A.dot(alpha[:, t]) * B[:, xt]
    return res

try:
    np.allclose(multiply_numpy_way(alpha, A, B, sequence), multiply_non_numpy_way(alpha, A, B, sequence))
except NameError:
    print('skip check')
    
def multiply_non_numpy_way_prev(prev_alpha, A, B, sequence):
    num_states = len(prev_alpha)
    
    res = np.zeros((num_states, len(sequence)))
    for t, xt in enumerate(sequence):
        for i in range(num_states):
            for j in range(num_states):
                res[i, t] += prev_alpha[j, 0] * A[i, j] * B[i, xt]
        prev_alpha = res[:, t].reshape(-1, 1)
        
    return res

def multiply_numpy_way_prev(prev_alpha, A, B, sequence):
    num_states = len(alpha)
    
    res = np.zeros((num_states, len(sequence)))
    for t, xt in enumerate(sequence):
        res[:, t] = A.dot(prev_alpha).flatten() * B[:, xt]
        prev_alpha = res[:, t].copy()
        
    return res

try:
    np.allclose(multiply_non_numpy_way_prev(prev_alpha, A, B, sequence), multiply_numpy_way_prev(prev_alpha, A, B, sequence))
except NameError:
    print('skip check')
    

def B_non_numpy_way(beta, A, B, sequence):
    num_states = len(beta)
    res = np.zeros((num_states, len(sequence)))
    
    for t, xt in enumerate(sequence):
        for i in range(num_states):
            for j in range(num_states):
                res[i, t] += beta[j, t] * A[j, i] * B[j, xt]
    return res

def B_numpy_way(beta, A, B, sequence):
    num_states = len(beta)
    res = np.zeros((num_states, len(sequence)))
    
    for t, xt in enumerate(sequence):
#         res[i, t] += beta[j,t] * A[j,i] * B[j, xt]
        res[:, t] = (beta[:,t] * B[:, xt]).dot(A)
    return res


try:
    assert(np.allclose(B_numpy_way(beta, A, B, sequence), B_non_numpy_way(beta, A, B, sequence)))
except NameError:
    print('skip check')

In [ ]:
# divide classes for clarity
class HMM(HMM_base):        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha, alpha_norms = self.forward(sequence)
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence, alpha_norms)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        gamma = (alpha * beta) / np.sum(alpha * beta, 0)# * (alpha_norms.T ** 2)
        # gamma is normalized -> no need to apply alpha_norms

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = alpha[:, t].reshape(-1, 1) * self.A * beta[:, t+1] * self.B[:, sequence[t+1]]
            xi += this_xi / np.sum(this_xi)
        
        # * alpha_norms[len(sequence)-1]
        log_likelihood = np.log(np.sum(alpha[:, len(sequence)-1]))
        return alpha, beta, gamma, xi, log_likelihood
    
    def forward(self, sequence, scale=False):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)))
        alpha_norms = np.zeros(len(sequence)) if scale else None
        
        for t, xt in enumerate(sequence):
            if t == 0:
                prev_alpha = self.pi * self.B[:, xt]
                alpha_initialized = True
            else:
#                 prev_alpha = self.A.T.dot(prev_alpha).flatten() * self.B[:, xt]
#                 prev_alpha = (self.A.T * prev_alpha).sum(axis=1) * self.B[:, xt]
                prev_alpha = (np.exp(np.log(self.A.T) + np.log(prev_alpha))).sum(axis=1)
                prev_alpha = np.log(prev_alpha) + np.log(self.B[:, xt])
                prev_alpha = np.exp(prev_alpha)
#                 prev_beta = prev_beta.sum(axis=1)
            
            alpha_norm = np.sum(prev_alpha)
            
            if scale:
                prev_alpha = prev_alpha / alpha_norm
                alpha_norms[t] = alpha_norm
                
            alpha[:, t] = prev_alpha.copy()
            
        
        assert(alpha_initialized)
        return alpha, alpha_norms
    
    def backward(self, sequence, alpha_norms=None):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        num_states = len(self.pi)
        beta = np.zeros((num_states, len(sequence)))
    
        for ind, xt in enumerate(sequence[::-1]):
            t = len(sequence) - ind - 1
            if ind == 0:
                prev_beta = np.ones(num_states)
                beta_initialized = True
            else:
#                 prev_beta = (prev_beta * self.B[:, xt_plus1]).dot(self.A.T)
                prev_beta = np.log1p(prev_beta) + np.log1p(self.B[:, xt_plus1]) + np.log1p(self.A)
                prev_beta = np.exp(prev_beta)
                prev_beta = prev_beta.sum(axis=1)
                
            
            if alpha_norms is not None:
                prev_beta = prev_beta / alpha_norms[t]
            beta[:, t] = prev_beta.copy()
            xt_plus1 = xt
    
        assert(beta_initialized)
        
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))


In [ ]:
# class HMM_cycles(HMM):
#     def forward(self, sequence):
#         # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
#         alpha = np.zeros((len(self.pi), len(sequence)))
        
#         num_states = len(self.pi)
#         for t, xt in enumerate(sequence):
#             if t == 0:
#                 alpha[:, 0] = self.pi * self.B[:, xt]
#                 alpha_initialized = True
#             else:
#                 for i in range(num_states):
#                     for j in range(num_states):
#                         alpha[i, t] += alpha[j, t - 1] * self.A[j, i] * self.B[i, xt]
    
#         assert(alpha_initialized)
        
#         return alpha
    
#     def backward(self, sequence):
#         # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
#         num_states = len(self.pi)
#         beta = np.zeros((num_states, len(sequence)))
        
#         for ind, xt in enumerate(sequence[::-1]):
#             t = len(sequence) - ind - 1
#             if ind == 0:
#                 beta[:, t] = 1.0
#                 beta_initialized = True
#             else:
#                 for i in range(num_states):
#                     for j in range(num_states):
#                         beta[i, t] += beta[j, t + 1] * self.A[i, j] * self.B[j, xt_plus1]
#             xt_plus1 = xt

#         assert(beta_initialized)
        
#         return beta
    
# # Since it's a substitution cipher we assume hidden states and observations have same alphabet.
# state_to_char_mapping = int_to_char_mapping
# observation_to_char_mapping = int_to_char_mapping

# # Initialize a HMM with the correct state/output spaces.
# hmm = HMM(observation_to_char_mapping, state_to_char_mapping)
# hmm_cycles = HMM_cycles(observation_to_char_mapping, state_to_char_mapping)

# hmm_cycles.pi = hmm.pi.copy()
# hmm_cycles.A = hmm.A.copy()
# hmm_cycles.B = hmm.B.copy()

# for sequence in tqdm(encrypted_sequences[:100]):
#     assert(np.allclose(hmm.forward(sequence), hmm_cycles.forward(sequence)))
#     assert(np.allclose(hmm.backward(sequence), hmm_cycles.backward(sequence)))